<a href="https://colab.research.google.com/github/Raynould-Joseph/Restricted-Blotzmann-Machine/blob/main/RBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q torch==1.0.0 torchvision

     |████████████████████████████████| 591.8 MB 542 bytes/s 
     |████████████████████████████████| 19.1 MB 1.2 MB/s 
     |████████████████████████████████| 19.1 MB 723 kB/s 
     |████████████████████████████████| 21.0 MB 1.5 MB/s 
     |████████████████████████████████| 23.2 MB 37.7 MB/s 
     |████████████████████████████████| 23.3 MB 1.2 MB/s 
     |████████████████████████████████| 23.3 MB 1.6 MB/s 
     |████████████████████████████████| 22.1 MB 1.2 MB/s 
     |████████████████████████████████| 22.1 MB 1.2 MB/s 
     |████████████████████████████████| 17.4 MB 71.2 MB/s 
     |████████████████████████████████| 17.3 MB 119 kB/s 
     |████████████████████████████████| 12.8 MB 45.9 MB/s 
     |████████████████████████████████| 12.7 MB 50.9 MB/s 
     |████████████████████████████████| 11.8 MB 43.9 MB/s 
     |████████████████████████████████| 5.9 MB 43.0 MB/s 
     |████████████████████████████████| 6.6 MB 49.0 MB/s 
     |████████████████████████████████| 6.6 MB 41.8 MB/s 
     

In [1]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/NLP lab/AmazonBooks - Sheet1csv.csv')
df

In [5]:
import numpy as np  
import pandas as pd  
import torch  
import torch.nn as nn  
import torch.nn.parallel  
import torch.optim as optim  
import torch.utils.data  
from torch.autograd import Variable  

In [6]:
# Importing the dataset  
movies = pd.read_csv('/content/gdrive/MyDrive/RBM/ml-1m/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1') 
movies

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [8]:
Ratings = pd.read_csv('/content/gdrive/MyDrive/RBM/ml-1m/ml-1m//ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')  
Ratings

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [10]:
# Preparing the training set and the test set  
training_set = pd.read_csv('/content/gdrive/MyDrive/RBM/ml-100k/ml-100k/u1.base', delimiter = '\t')  
training_set

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561
...,...,...,...,...
79994,943,1067,2,875501756
79995,943,1074,4,888640250
79996,943,1188,3,888640250
79997,943,1228,3,888640275


In [12]:
training_set = np.array(training_set, dtype = 'int')  
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [13]:
test_set = pd.read_csv('/content/gdrive/MyDrive/RBM/ml-100k/ml-100k/u1.test', delimiter = '\t')  

In [14]:
test_set = np.array(test_set, dtype = 'int')  

In [15]:
# Getting the number of users and movies  
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))  

In [16]:
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))  

In [17]:
max(training_set[:,1])  


1682

In [18]:
max(test_set[:,1])  

1591

In [22]:
# Converting the data into an array with users in lines and movies in columns  
def convert(data):  
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:,1][data[:,0] == id_users]
    id_ratings = data[:,2][data[:,0] == id_users] 
    ratings = np.zeros(nb_movies)  
    ratings[id_movies - 1] = id_ratings  
    new_data.append(list(ratings))
    return new_data  
    training_set = convert(training_set)  
    test_set = convert(test_set)  
    
  

In [24]:
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [25]:
# Converting the data into Torch tensors  
training_set = torch.FloatTensor(training_set)  

In [26]:
test_set = torch.FloatTensor(test_set)  


In [27]:
# Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)  
training_set[training_set == 0] = -1  

In [28]:
training_set[training_set == 1] = 0  
training_set[training_set == 2] = 0  


In [29]:
training_set[training_set >= 3] = 1  

In [30]:
test_set[test_set == 0] = -1  
test_set[test_set == 1] = 0  
test_set[test_set == 2] = 0  
test_set[test_set >= 3] = 1  

In [31]:
# Creating the architecture of the Neural Network  
class RBM():  
    def __init__(self, nv, nh):  
      self.W = torch.randn(nh, nv)
      self.a = torch.randn(1, nh)    
      self.b = torch.randn(1, nv)  
      def sample_h(self, x): 
        wx = torch.mm(x, self.W.t())   
        activation = wx + self.a.expand_as(wx)  
        p_h_given_v = torch.sigmoid(activation) 
        return p_h_given_v, torch.bernoulli(p_h_given_v)   
      


In [59]:
def train(self, v0, vk, ph0, phk):  
  self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk) 
  self.b += torch.sum((v0 - vk), 0)  
  self.a += torch.sum((ph0 - phk), 0)  

  

In [60]:
nv = len(training_set[0])  
nh = 100  
batch_size = 100  
rbm = RBM(nv, nh)

In [61]:
# Training the RBM  
nb_epoch = 10  
 



In [62]:
for epoch in range(1, nb_epoch + 1):  
  train_loss = 0  
  s = 0. 

In [64]:
for id_user in range(0, nb_users - batch_size, batch_size):  
  vk = training_set[id_user:id_user+batch_size] 
  v0 = training_set[id_user:id_user+batch_size]  


In [66]:
ph0,_ = rbm.sample_h(v0)  

AttributeError: ignored

In [67]:

for k in range(10): 
  _,hk = rbm.sample_h(vk) 
  _,vk = rbm.sample_v(hk)  
  vk[v0<0] = v0[v0<0]  
  phk,_ = rbm.sample_h(vk)  
  rbm.train(v0, vk, ph0, phk)  
  train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))  
  s += 1. 
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))  

AttributeError: ignored